In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/preprocessed_review.csv")

In [3]:
def label_review(rate):
    if rate >= 4: return 'positive'
    elif rate == 3: return 'neutral'
    else: return 'negative'

df['sentiment'] = df['Puan'].apply(label_review)
df['sentiment'].value_counts()

sentiment
positive    45656
negative    18419
neutral      2219
Name: count, dtype: int64